You could just run the following notebook without hesitation

The following block are the functions to judge whether a graph has a bipartite

In [19]:
perms = [[0,4,8],[0,5,7],[1,5,6],[1,3,8],[2,4,6],[2,3,7]]
def bipartite(s):
    for indices in perms:
        if s[indices[0]]=="1" and s[indices[1]]=="1" and s[indices[2]]=="1":
            return True
    return False

def bipartite_with_unknown(s):
    # Return True/False if there is already a matching or there could never be a matching
    # Return None if there are both probabilities
    good_s=s.replace("x","1")
    bad_s=s.replace("x","0")
    if bipartite(bad_s): return True
    if bipartite(good_s): return None
    return False

In [20]:
import json
char="01x"
all_graph=set() # all possible graph
for i in range (3**9):
    s=""
    n=i
    for j in range(9):
        s+=char[n%3]
        n=n//3
    all_graph.add(s)
classes=[]

char="01"
all_graph_complete=set() # all possible complete graph. The complete is for "without unknown edges"
for i in range (2**9):
    s=""
    n=i
    for j in range(9):
        s+=char[n%2]
        n=n//2
    all_graph_complete.add(s)
classes_complete = []

In [21]:
perm=[[0,1,2],[0,2,1],[1,2,0],[1,0,2],[2,1,0],[2,0,1]]

all_possible_perms = []
for i in perm:
    for j in perm:
        equiv=""
        mapping={}
        for k in range(3):
            for l in range(3):
                mapping[k*3+l]=i[k]*3+j[l]
        all_possible_perms.append(mapping)
for i in perm:
    for j in perm:
        equiv=""
        mapping={}
        for k in range(3):
            for l in range(3):
                mapping[k+3*l]=i[k]*3+j[l]
        all_possible_perms.append(mapping)

def equiv_class(s):
    # It returns a mappings. 
    # The keys are all the isomorphic graphs, 
    # and the value associated with each key is the mapping FROM canonical s TO the key
    ans={}
    for perm in all_possible_perms:
        equiv = ""
        for ind in range(9):
            equiv += s[perm[ind]]
        perm2 = {v:k for k,v in perm.items()}
        ans[equiv] = perm2
    return ans

In [22]:
map_from_canonical={}
# Draw a graph as canonical from each class of isomirphic type.
# The dictionary above is canonical graph for each graph
while len(all_graph)!=0:
    for graph in all_graph:
        break
    class_of_graph = equiv_class(graph)
    for equiv in class_of_graph.keys():
        map_from_canonical[equiv] = class_of_graph[equiv]
    classes.append(list(class_of_graph.keys()))
    all_graph -= set(class_of_graph)

while len(all_graph_complete)!=0:
    for graph in all_graph_complete:
        break
    class_of_graph = equiv_class(graph)
    classes_complete.append(list(class_of_graph.keys()))
    all_graph_complete -= set(class_of_graph)
    
canonicals = {}
types = set() #distinct types of graph
for graphs in classes:
    canonical = graphs[0]
    types.add(canonical)
    for graph in graphs:
        canonicals[graph] = canonical

In [23]:
def guess(graph, algorithm):
    # Given anAlgorithm written in the canonical graphs, And we have the query graph.
    # Return the index.
    canon = canonicals[graph]
    ask_in_canon = algorithm[canon]
    return map_from_canonical[graph][ask_in_canon]

def evaluation(algorithm):
    # For all the complete graphs, calculate the expected yes and no to query this.
    # Return the max expected number of yes and no for all graph, For all the graphs with and without matching.
    # Calculate separately if the graph itself is containing/not containing a matching.
    yes4yes = 0
    crit_yes4yes = None
    no4yes = 0
    crit_no4yes = None
    yes4no = 0
    crit_yes4no = None
    no4no = 0
    crit_no4no = None
    for graphs in classes_complete:
        avg_num = len(graphs)
        yes = 0
        no = 0
        for graph in graphs:
            begin = "x"*9
            for _ in range(9):
                if bipartite_with_unknown(begin)!=None:
                    break
                guessing = guess(begin, algorithm)
                if graph[guessing] == "0": no+=1
                else: yes+=1
                begin = begin[:guessing]+graph[guessing]+begin[guessing+1:]
        avg_yes = yes/avg_num
        avg_no = no/avg_num
        if bipartite_with_unknown(graphs[0]) == True:
            if avg_yes > yes4yes:
                yes4yes = avg_yes
                crit_yes4yes = graphs[0]
            if avg_no > no4yes:
                no4yes = avg_no
                crit_no4yes = graphs[0]
        if bipartite_with_unknown(graphs[0]) == False:
            if avg_yes > yes4no:
                yes4no = avg_yes
                crit_yes4no = graphs[0]
            if avg_no > no4no:
                no4no = avg_no
                crit_no4no = graphs[0]
    return (yes4yes, crit_yes4yes, no4yes, crit_no4yes, yes4no, crit_yes4no, no4no, crit_no4no)

In [24]:
# de-duplicate
valid = {}
for graph in types:
    valid_op = {}
    reached = set()
    if bipartite_with_unknown(graph)==None:
        ind = [i for i in range (9) if graph[i]=="x"]
        for i in ind:
            canonical_0 = canonicals[graph[:i]+"0"+graph[i+1:]]
            canonical_1 = canonicals[graph[:i]+"1"+graph[i+1:]]
            if (canonical_0,canonical_1) not in reached:
                valid_op[i]=(canonical_0,canonical_1)
                reached.add((canonical_0,canonical_1))
        valid[graph]=valid_op
    else:
        valid[graph]={}

# branching the step with splitting T/F
valid2 = {}
okay = []
for graph in valid.keys():
    valid_op = valid[graph]
    flag = True
    if len(valid_op)>=2:
        for i in valid_op.keys():
            if bipartite_with_unknown(valid_op[i][0])!=None and bipartite_with_unknown(valid_op[i][1])!=None:
                if flag:
                    okay.append(graph)
                    flag = False
                    valid2[graph]={i : valid_op[i]}
    if flag:
        valid2[graph]=valid[graph]

In [25]:
'''
The following is a depreciated branching method
for graph in valid2.keys():
    valid_op = valid2[graph]
    True_OK=False
    True_OK_ind=None
    False_OK=False
    False_OK_ind=None
    True_UNK=False
    False_UNK=False
    if len(valid_op)>=2:
        for i in valid_op.keys():
            config0=bipartite_with_unknown(valid_op[i][0])
            config1=bipartite_with_unknown(valid_op[i][1])
            if config1==True:
                True_UNK = True
                if valid_op[i][0] in okay:
                    True_OK = True
                    True_OK_ind = i
            if config0==False:
                False_UNK=True
                if valid_op[i][1] in okay:
                    False_OK = True
                    False_OK_ind = i
        if True_OK==True and False_UNK==False:
            valid2[graph]={True_OK_ind:valid_op[True_OK_ind]}
        elif False_OK==True and not True_UNK==True:
            valid2[graph]={False_OK_ind:valid_op[False_OK_ind]}
'''

'\nThe following is a depreciated branching method\nfor graph in valid2.keys():\n    valid_op = valid2[graph]\n    True_OK=False\n    True_OK_ind=None\n    False_OK=False\n    False_OK_ind=None\n    True_UNK=False\n    False_UNK=False\n    if len(valid_op)>=2:\n        for i in valid_op.keys():\n            config0=bipartite_with_unknown(valid_op[i][0])\n            config1=bipartite_with_unknown(valid_op[i][1])\n            if config1==True:\n                True_UNK = True\n                if valid_op[i][0] in okay:\n                    True_OK = True\n                    True_OK_ind = i\n            if config0==False:\n                False_UNK=True\n                if valid_op[i][1] in okay:\n                    False_OK = True\n                    False_OK_ind = i\n        if True_OK==True and False_UNK==False:\n            valid2[graph]={True_OK_ind:valid_op[True_OK_ind]}\n        elif False_OK==True and not True_UNK==True:\n            valid2[graph]={False_OK_ind:valid_op[False_

In [26]:
print(len([i for i in valid2.keys() if len(valid2[i])>1]))

169


In [27]:
selections = valid2
number_of_known={i:set() for i in range(10)}
travelling_list = []
for graph in types:
    number_of_known[graph.count("x")].add(graph)
for i in range(10):
    travelling_list+=number_of_known[i]

yes_vs_no={} # {graph:(number of yes, number of no)}
for graph in travelling_list:
    if "x" not in graph:
        yes_vs_no[graph] = (1,0) if bipartite_with_unknown(graph) else (0,1)
    else:
        ind = graph.index("x")
        yes_graph = canonicals[graph[:ind]+"1"+graph[ind+1:]]
        no_graph = canonicals[graph[:ind]+"0"+graph[ind+1:]]
        total_yes = yes_vs_no[yes_graph][0]+yes_vs_no[no_graph][0]
        total_no = yes_vs_no[yes_graph][1]+yes_vs_no[no_graph][1]
        yes_vs_no[graph] = (total_yes, total_no)

pre_algorithm = {} # First rule out all the selections with smaller number of true graphs if the query edge is true.
for graph in valid2.keys():
    max_diff = -1
    max_indices = set()
    selection = valid2[graph]
    for index in selection.keys():
        go_to = selection[index]
        yes_graph = go_to[1]
        no_graph = go_to[0]
        yes_for_yes = yes_vs_no[yes_graph][0]
        yes_for_no = yes_vs_no[no_graph][0]
        if abs(yes_for_yes-yes_for_no) > max_diff:
            max_indices = {index}
            max_diff = abs(yes_for_yes-yes_for_no)
        elif abs(yes_for_yes-yes_for_no) == max_diff:
            max_indices.add(index)
    pre_algorithm[graph] = {i:selection[i] for i in max_indices}

In [28]:
# All the possible algorithms.
total_algo = {i:[] for i in types}
for graph in travelling_list:
    if pre_algorithm[graph] == {}:
        total_algo[graph] = set()
    else:
        option = pre_algorithm[graph]
        for i in option.keys():
            yes_to = option[i][0]
            no_to = option[i][1]
            yes_algo = total_algo[yes_to]
            no_algo = total_algo[no_to]
            if len(yes_algo)==0 and len(no_algo)==0:
                total_algo[graph].append({graph:i})
            elif len(yes_algo)==0 or len(no_algo)==0:
                for algo in list(yes_algo)+list(no_algo):
                    init={graph:i}
                    init.update(algo)
                    total_algo[graph].append(init)
            else:
                for y_algo in yes_algo:
                    for n_algo in no_algo:
                        init={graph:i}
                        init.update(y_algo)
                        init.update(n_algo)
                        total_algo[graph].append(init)
        #print("graph",graph,"has",len(algorithms[graph]),"possible algorithms")
#print("finish")

In [29]:
len(total_algo["xxxxxxxxx"])

512

In [30]:
'''
A deppreciated method to calculate one of the algorithm.
algorithm = {}
cur = {"x"*9}
for i in range(10):
    aux = set()
    for graph in cur:
        selection = selections[graph]
        if len(selection)>0:
            max_diff = -1
            max_index = -1
            for index in selection.keys():
                go_to = selection[index]
                yes_graph = go_to[1]
                no_graph = go_to[0]
                yes_for_yes = yes_vs_no[yes_graph][0]
                yes_for_no = yes_vs_no[no_graph][0]
                if abs(yes_for_yes-yes_for_no) > max_diff:
                    max_index = index
                    max_diff = abs(yes_for_yes-yes_for_no)
            algorithm[graph] = int(max_index)
            aux|=set(selection[max_index])
    cur = aux

evals = evaluation(algorithm)
yes4yes = evals[0]
no4yes = evals[2]
yes4no = evals[4]
no4no = evals[6]
import math
print((yes4yes + no4no)/2 + math.sqrt((yes4yes - no4no)**2/4 + no4yes*yes4no))
'''

'\nA deppreciated method to calculate one of the algorithm.\nalgorithm = {}\ncur = {"x"*9}\nfor i in range(10):\n    aux = set()\n    for graph in cur:\n        selection = selections[graph]\n        if len(selection)>0:\n            max_diff = -1\n            max_index = -1\n            for index in selection.keys():\n                go_to = selection[index]\n                yes_graph = go_to[1]\n                no_graph = go_to[0]\n                yes_for_yes = yes_vs_no[yes_graph][0]\n                yes_for_no = yes_vs_no[no_graph][0]\n                if abs(yes_for_yes-yes_for_no) > max_diff:\n                    max_index = index\n                    max_diff = abs(yes_for_yes-yes_for_no)\n            algorithm[graph] = int(max_index)\n            aux|=set(selection[max_index])\n    cur = aux\n\nevals = evaluation(algorithm)\nyes4yes = evals[0]\nno4yes = evals[2]\nyes4no = evals[4]\nno4no = evals[6]\nimport math\nprint((yes4yes + no4no)/2 + math.sqrt((yes4yes - no4no)**2/4 + no4y

In [31]:
import math
# Calculate the best algorithm and its eigenvalue.
min_eigen = 9
best_algo = None
for algo in total_algo["x"*9]:
    evals = evaluation(algo)
    yes4yes = evals[0]
    no4yes = evals[2]
    yes4no = evals[4]
    no4no = evals[6]
    eigen = (yes4yes + no4no)/2 + math.sqrt((yes4yes - no4no)**2/4 + no4yes*yes4no)
    if eigen < min_eigen:
        best_algo = algo
        min_eigen = eigen

In [32]:
min_eigen

5.9816453557597065

In [33]:
best_algo

{'xxxxxxxxx': 0,
 'xxx0xxxxx': 0,
 'xxxx0xx0x': 1,
 '1xx0xx0xx': 4,
 '001xxx0xx': 7,
 '0x11xx0x0': 7,
 '0011xxxxx': 7,
 '0xx010xx1': 2,
 'x1xx10100': 2,
 'xx1xx0xxx': 3,
 'x10xxxxx0': 5,
 '1x0xx1xxx': 7,
 'xx1x0x1x0': 5,
 'xx01xx011': 1,
 'x1010xx10': 5,
 '001xx1110': 3,
 'x01110001': 0,
 'xxxx1xxxx': 0,
 'x0xxxx1xx': 2,
 '1xxxxxx00': 6,
 'x1xxxx010': 0,
 '11x0xx00x': 4,
 '001011xxx': 6,
 'x0xx11x01': 0,
 'xx011x100': 1,
 '1x100110x': 1,
 '1xxxx10xx': 7,
 '1xxx00xx1': 3,
 '100xx11xx': 7,
 '0x001x1x1': 5,
 '10xx11100': 2,
 'xxx1xxxx1': 1,
 'x1x1xxxx0': 2,
 '110x11x00': 6,
 'x11xx01xx': 4,
 '0x1x1x0x1': 3,
 '11x001xx1': 6}

In [34]:
evaluation(best_algo)

(3.6666666666666665,
 '100110111',
 1.6944444444444444,
 '101100011',
 2.5555555555555554,
 '001110001',
 4.111111111111111,
 '001110001')